<img src="./logo_UTN.svg" align="right" width="150" /> 

#### Teoría de Circuitos II - R4001

# TAREA SEMANAL 12

Autor: Nicolás Burgos


## ENUNCIADO

### PARTE A:

Diseñe el cuadripolo A para que, respetando la siguiente topología, se comporte como:

$\rightarrow$ Filtro pasa bajos Cheby de 3er. orden.

$\rightarrow$ No disipativo.

$\rightarrow$ Normalizado en frecuencia e impedancia.

![alt text](topologia_ts12.png)

1. Obtenga la impedancia de entrada al cuadripolo A, cargado con un resistor de $1 \Omega$ a la salida.

2. Sintetice **A** como un cuadripolo escalera.

3. Simule el comportamiento de la red en *LTspice* graficando $S_{21}$ y $S_{11}$ en función de la frecuencia.

4. Explique el comportamiento de A, a partir de los valores de $S_{11}$ en las siguientes frecuencias: **centro de la banda de paso, frecuencia de corte, y banda de transición y centro de la banda de detenida**.

5. Modifique el circuito para que la frecuencia de corte sea $2 \pi 10^6$ rad/s y la resistencia del generador sea $50 \Omega$.

### PARTE B:

Trabajo de Investigación sobre NANO VNA ( Vector Network Analyzer ):

1. Explique brevemente la función principal de un VNA y en que situación/es puede ser fundamental tenerlo como herramienta de medición.

2. Comparar respecto a un DSO GW Insteak características de: Ancho de Banda, Rango Dinámico e Impedancia de Entrada.

3. Además de las mediciones de los parametros  , investigar si el equipo cuenta con alguna medición en el dominio del tiempo.

### RESOLUCIÓN

#### PARTE A:

**(1)** En base a lo especificado sobre las características que debe poseer el cuadripolo A, buscaremos imponer el parámetro de transmisión $S_{21}$, y así restringir el de reflexión $S_{11}$, que nos ayudará a obtener la impedancia de entrada $Z_1$:

\begin{equation}
|S_{21}|^2 = \frac{1}{1 + C_n^2} = \frac{1}{1 + C_3^2} = \frac{1}{1 + (4 \omega^3 - 3 \omega)^2} = \frac{1}{16 \omega^6 - 24 \omega^4 + 9 \omega^2 + 1}
\end{equation}

Podemos observar que, en un filtro no disipativo, se da la siguiente relación entre transmisión y reflexión:

\begin{equation}
|S_{11}|^2 = 1 - |S_{21}|^2
\end{equation}

De esta forma, podemos obtener lo siguiente:

\begin{equation}
|S_{11}|^2 = 1 - \frac{1}{16 \omega^6 - 24 \omega^4 + 9 \omega^2 + 1}
\end{equation}

\begin{equation}
\boxed{|S_{11}(j\omega)|^2 = \frac{16 \omega^6 - 24 \omega^4 + 9 \omega^2}{16 \omega^6 - 24 \omega^4 + 9 \omega^2 + 1}}
\end{equation}

Para encontrar el parámetro de reflexión en la entrada $S_{11}$ lo que haremos será obtener la expresión de $|S_{11}(s)|^2$ y buscar sus polos y ceros de forma computacional, según en código presentado más abajo:

\begin{equation}
|S_{11}(s)|^2 = |S_{11}(j\omega)|^2|_{\omega = \frac{s}{j}} = \frac{(-16) s^6 + (-24) s^4 + (-9) s^2}{(-16) s^6 + (-24) s^4 + (-9) s^2 + 1}
\end{equation}

In [1]:
import numpy as np
import scipy.signal as signal

s11_mod_num = [-16., 0., -24., 0., -9., 0., 0.]
s11_mod_den = [-16., 0., -24., 0., -9., 0., 1.]

roots_s11_num = np.roots(s11_mod_num)
roots_s11_den = np.roots(s11_mod_den)

print('Ceros de |S11|^2')
print(roots_s11_num)

print('------------------------------------')

print('Polos de |S11|^2')
print(roots_s11_den)

Ceros de |S11|^2
[-1.81146764e-12+0.86602541j -1.81146764e-12-0.86602541j
  1.81159254e-12+0.86602539j  1.81159254e-12-0.86602539j
  0.00000000e+00+0.j          0.00000000e+00+0.j        ]
------------------------------------
Polos de |S11|^2
[-0.14901791+0.90366975j -0.14901791-0.90366975j  0.14901791+0.90366975j
  0.14901791-0.90366975j -0.29803582+0.j          0.29803582+0.j        ]


Con estos resultados en mano, solo nos quedaremos con las raíces que estén en el semiplano izquierdo y una en el origen, pudiendo obtener la expresión completa de $S_{11}$:

In [2]:
print('Numerador de S11')

num_s11 = np.convolve( [1, roots_s11_num[5]], np.convolve( [1, roots_s11_num[2]], [1, roots_s11_num[3]] ) )
print(num_s11)

print('------------------------------------')

print('Denominador de S11')

den_s11 = np.convolve( [1, roots_s11_den[5]], np.convolve( [1, roots_s11_den[2]], [1, roots_s11_den[3]] ) )
print(den_s11)

Numerador de S11
[1.00000000e+00+0.j 3.62318509e-12+0.j 7.49999982e-01+0.j
 0.00000000e+00+0.j]
------------------------------------
Denominador de S11
[1.        +0.j 0.59607164+0.j 0.9276507 +0.j 0.25      +0.j]


Construyendo la expresión de $S_{11}$ de forma prolija, nos queda:

\begin{equation}
S_{11}(s) = \frac{s^3 + 0.75 s}{s^3 + 0.596 s^2 + 0.928 s + 0.25}
\end{equation}

Con este resultado, solo nos queda aplicar la ecuación relacionada a este parámetro de reflexión para encontrar la impedancia de entrada de la red a sintetizar despues:

\begin{equation}
S_{11}(s) = \frac{Z_1(s) - R_{01}}{Z_1(s) + R_{01}} = \frac{Z_1(s) - 1}{Z_1(s) + 1}
\end{equation}

\begin{equation}
Z_1(s) = \frac{S_{11}(s) - 1}{1 - S_{11}(s)}
\end{equation}

\begin{equation}
\boxed{Z_1(s) = \frac{2 s^3 + 0.596 s^2 + 1.678 s + 0.25}{0.596 s^2 + 0.178 s + 0.25}}
\end{equation}

$\space$

**(2)** Para poder sintetizar el cuadripolo pedido, lo haremos a partir de la impedancia de entrada $Z_1(s)$ encontrada en el inciso anterior. Para ello, aplicaremos remociones continuas en infinito por el método de Cauer, por lo que será conveniente tener a la mano la siguiente expresión, donde *P* y *Q* son dos polinomios que representan numerador y denominador respectivamente, *C* es el cociente y *R* el resto de las divisiones.

\begin{equation}
F = \frac{P}{Q} = C + \frac{1}{\frac{Q}{R}}
\end{equation}

Buscaremos realizar la división entre P y Q, y luego proceder a dividir Q por el resto R obtenido, y así sucesivamente hasta llegar a resto cero. Así, podemos llegar al siguiente resultado:

\begin{equation}
Z_1(s) =  \frac{2 s^3 + 0.596 s^2 + 1.678 s + 0.25}{0.596 s^2 + 0.178 s + 0.25} = 3.356s + \frac{1}{ \frac{0.596 s^2 + 0.178 s + 0.25}{0.839 s + 0.25} } = 3.356s + \frac{1}{ 0.710s + \frac{1}{ \frac{0.839 s + 0.25}{0.25} } } = 3.356s + \frac{1}{ 0.710s + \frac{1}{ 3.356s + 1 } }
\end{equation}

Aclarando un poco los resultados, la red escalera quedará como una red Tee, con dos inductores en serie (ambos de valor 3.356 por ser una red simétrica) y un capacitor en derivación (de valor 0.710). El circuito resultante es el siguiente:

![alt text](circ_res_a2.png)

$\space$

**(3)** El circuito utilizado para la simulación fue el siguiente, y las gráficas de los parámetros solicitados se ven a continuación:

![alt text](circ_sim_a3.png)

![alt text](sim_param_s21.png)

![alt text](sim_param_s11.png)

![alt text](sim_s21_s11.png)

En la simulación es posible observar que se logró cumplir con las expresiones de los parámetros solicitadas y encontradas. Podemos ver que el parámetro $S_{21}$ efectivamente se comporta como un filtro pasa bajos no disipativo, y como su correlato, el parámetro $S_{11}$ resulta comportarse como un filtro pasa altos Notch.

Estos comportamientos nos determinan que tan adaptada está la red para distintos valores de frecuencia. Viendo el parámetro $S_{11}$, en los puntos donde la magnitud, medida en dB, cae a valores cada vez más negativos, la red se encontrará en sus mejores condiciones de adaptación. Por otro lado, cuanto más cerca de 0 dB se encuentre, más desadaptada estará.

$\space$

**(4)** Por medio del parámetro $S_{11}$, podremos determinar el comportamiento del cuadripolo para distintos valores de frecuencia. Graficando simultáneamente este parámetro junto con $S_{21}$, cuyo comportamiento es el de un filtro pasa bajos, y seleccionando puntos del espectro de $S_{21}$, podremos observar lo siguiente:

**$\rightarrow$ CENTRO DE LA BANDA DE PASO DE $S_{21}$:** en el centro de la banda de paso, es decir, en baja frecuencia, veremos que la magnitud del parámetro $S_{11}$ se encuentra en -34.5 dB. Esto implica muy buena adaptación de la red, con muy poca reflexión en la banda donde se espera que la señal pase.

**$\rightarrow$ FRECUENCIA DE CORTE:** la frecuencia de corte la podemos encontrar en el punto donde el módulo graficado cae a -3 dB aproximadamente. Dicho valor de frecuencia, debido al diseño del filtro, será $\omega_c = 1$ rad/seg, lo que equivale a fc = 0.159 Hz. Allí, comienza la Banda de Stop del “filtro” que representa $S_{21}$, pero se puede observar que, debido a que tiene el mismo valor de magnitud, $S_{11}$ indica ahora una gran desadaptación de la red, reflejando la potencia transmitida casi por completo.

**$\rightarrow$ BANDA DE TRANSICIÓN Y CENTRO DE LA BANDA DE DETENIDA:** previo al valor de la frecuencia de corte, donde inicia la Banda de Transición, es posible observar un valle en forma de aguja en $S_{11}$, lo que indica que para el valor máximo de potencia que deja pasar el “filtro” que representa $S_{21}$, se da un punto de adaptación de la red, de igual forma a lo explicado antes, y posteriormente se comienza a reflejar progresivamente la potencia transmitida. Por otro lado, en el centro de la Banda de Detenida, es decir, en alta frecuencia, se observa que $S_{21}$ cae en magnitud, y $S_{11}$ sube a 0 dB, lo cual nos indica una reflexión total de la potencia, ya que el filtro LC la rechaza completamente, generando una situación de total desadaptación. Como no puede disiparla, la energía de devuelve por completo a la fuente.

$\space$

**(5)** Para lograr una $\omega_c' = 2 \pi 10^6$ rad/seg y $R_g' = R_L´= 50 \Omega$ se debe realizar un escalado de los componentes tanto en impedancia como en frecuencia. Para ello, tenemos que determinar las normas necesarias:

\begin{equation}
\Omega_Z = \frac{R_g'}{R_g} = \frac{R_L'}{R_L} = 50 \Omega
\end{equation}

\begin{equation}
\Omega_\omega = \frac{\omega_c'}{\omega_c} = 2 \pi 10^6
\end{equation}

Con estos valores, escalamos los componentes antes hallados:

\begin{equation}
L_1' = L_1 \cdot \frac{\Omega_Z}{\Omega_\omega} = 26.706 \ \mu Hy
\end{equation}

\begin{equation}
L_2' = L_2 \cdot \frac{\Omega_Z}{\Omega_\omega} = 26.706 \ \mu Hy
\end{equation}

\begin{equation}
C_1' = \frac{C_1}{\Omega_Z \cdot \Omega_\omega} = 2.260 \ nF
\end{equation}

Al simular el circuito de antes con los valores nuevos, obtenemos lo siguiente:

![alt text](circ_sin_a5.png)

![alt text](sim_a5.png)

#### PUNTO B:

**(1)** Un VNA (Vector Network Analizer) es un instrumento que mide las características de una red eléctrica, como antenas o cables, enviando una señal y analizando lo que regresa.

Su función principal es caracterizar el comportamiento de componentes de radiofrecuencia (RF), midiendo cómo interactúan con una señal eléctrica. Desde la teoría explicada en clase, podemos decir que, una vez enviada esa señal, el VNA evalúa dos fenómenos físicos fundamentales:

**$\rightarrow$ Reflexión**: se determina cuánta señal “rebota” a intentar entrar en el dispositivo del otro lado de la línea. Esto nos ayuda a determinar el parámetro $S_{11}$.

**$\rightarrow$ Transmisión**: se determina cuánta señal logra “cruzar” el dispositivo hasta el otro lado. Con ello, se logra medir el parámetro $S_{21}$.

La diferencia con otros equipos de medición, es que un VNA permite medir magnitud, es decir, cuán fuerte es la señal, y por otro lado también fase, cuánto se ha retrasado o desplazado la onda en el tiempo. Esto permite hacer uso del Diagrama de Smith.

Existe una amplia gama de situaciones donde este dispositivo puede ser de gran ayuda. Algunas de ellas pueden ser las siguientes:

*(A)* Ajuste de antenas, intentando encontrar la frecuencia exacta de resonancia, minimizando la ROE o Relación de Onda Estacionaria.

*(B)* Caracterizar filtros, pudiendo ver exactamente qué frecuencias dejan pasar y cuáles bloquean, pudiendo determinar su ancho de banda.

*(C)* Medir cables, pudiendo determinar la pérdida de un cable o incluso su longitud exacta, por medio de la fase para determinar el retardo de grupo.

$\space$

**(2)** La principal diferencia entre un **VNA** y un **DSO GW Instek** es el dominio en el que cada uno trabaja. Mientras que el DSO trabaja en el **Dominio del Tiempo**, mostrando cómo cambia el voltaje segundo a segundo, un VNA trabaja en el **Dominio de la Frecuencia**, mostrando cómo responde un componente a distintas frecuencias.

Luego, si nos metemos en especificaciones técnicas, podemos hacer la siguiente comparación:

**$\rightarrow$ Ancho de Banda**: En un DSO el ancho de banda es un límite analógico pasa bajos, siendo de banda ancha instantánea. En un VNA, el ancho de banda puede ser mucha mayor, ya que, para expandirlo, suelen utilizar el método **Harmonic Mixing** o **Mezcla por Armónicos**, midiendo señales cuadradas que generan múltiplos de la frecuencia fundamental, lo cual puede generarle imprecisión en las mediciones para altas frecuencias por el ruido.

**$\rightarrow$ Rango Dinámico**: Esto puede definirse como la diferencia entre la señal más fuerte que es posible medir sin distorsión, y la más débil que puede distinguirse del ruido de fondo o piso de ruido.

Por un lado, un DSO se encuentra limitado por su Conversor Analógico Digital (ADC), que por lo general es de 8 bits, y por su rango dinámico libre de espurios que suele rondar los 40-50 dB, haciendo imposible detectar una señal pequeña montada sobre otra mucho más grande o bien medir un filtro con rechazo profundo.

Por otro lado, un VNA posee un receptor de banda estrecha, con lo que puede saber exactamente qué frecuencia está transmitiendo, sintonizando su receptor solo a esa frecuencia e ignorando el ruido en el resto, obteniendo un rango dinámico típico de 60 a 90 dB. Como resultado, es capaz de medir filtros con grandes atenuaciones, sin problemas.

**$\rightarrow$ Impedancia de Entrada**: Un dispositivo DSO cuenta con una impedancia de entrada alta, del orden de $1 M\Omega$, diseñada para no cargar el circuito bajo prueba, siendo ideal para medir voltajes en una placa base o un microcontrolador, sin que la corriente se drene al instrumento.

Por otra parte, un VNA posee una impedancia de entrada, y también de salida, bajas de $50 \Omega$, ya que es fundamental la adaptación en sus mediciones. Esto genera que no puedan utilizarse para medir circuitos de alta impedancia, ya que drenará toda la corriente y señal, colapsando el voltaje y arrojando mediciones erróneas. Se encuentra diseñado para sistemas de transmisión de potencia, como cables coaxiles o antenas.

$\space$

**(3)** Aunque se especificó más arriba que un VNA trabaja en el dominio de la frecuencia, los más modernos, incluido en NanoVNA, tienen una función para mostrar resultados en el dominio del tiempo.

Esto lo hace por medio de realizar un barrido en frecuencia normal, midiendo magnitud y fase en diversos puntos, y luego le aplica a los datos obtenidos la **Transformada Inversa de Fourier**. De esta forma, logra simular la respuesta al impulso o al escalón en el dominio de tiempo a partir de los datos de reflexión obtenidos en frecuencia.

Al ser una medición calculada y no nativa, la precisión dependerá del barrido en frecuencia. De esta forma, utilizando altas frecuencias se podrá medir una mayor separación en tiempo, y viceversa.

Además, posee modos para simular la respuesta al impulso en modo pasabanda o modo pasa bajos, y cuenta con la capacidad de acotar el rango de frecuencias a medir por medio de ventanas, específicamente de Kaiser o rectangulares, permitiendo obtener el máximo rango dinámico y la mayor resolución, respectivamente.